# Model Training and Deployment Pipeline

The pipeline is aiming to meet below needs:
* Data source change to trigger model retraining and deployment
* Model training code change to trigger model retraining and deployment
* Define a process to evaluate trained model and promotion to production environment

To achieve the needs, we design the pipeline with AWS services:
* Step Functions with [Step Functions Data Science SDK v2.0.0rc1](https://aws-step-functions-data-science-sdk.readthedocs.io/en/v2.0.0rc1/) to orchestrate model training and deployment in Amazon SageMaker platform.
* AWS CodePipeline to define the high-level orchestration from source code and data source changes triggering model training & deployment on Dev and Production environments
* AWS CodeBuild to process model training and deployment workflow creation and data source update.
* AWS Lambda to process basic functions in model training / deployment
* AWS Simple Notification Service to process notification.
* AWS CloudFormation to create the demo ML pipeline stack.

Below is the design diagram:
> Pending...


## Pipeline Demo

We demo how to trigger source code and data source change, which will trigger ML Pipeline build.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pipeline.ml_pipeline_dependencies import *

### Submit Data Source Change

Download source file

In [ ]:
!wget https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip

In [ ]:
import zipfile
with zipfile.ZipFile("./bank-additional.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

Upload data source to target S3 location.

In [ ]:
ssm = boto3.client('ssm')
response = ssm.get_parameter(Name = "/ml_pipeline/pipeline_artifact_s3_bucket_name")
pipeline_artifact_bucket_name = response['Parameter']['Value']
model_name = 

In [ ]:
data_source_file = "./bank-additional/bank-additional-full.csv"

s3://${pipeline_artifact_bucket_name}/preprocessing/input/bank-additional-full.csv
    
sagemaker.s3.S3Uploader.upload(
    data_source_file, 
    processing_input_path, 
    sagemaker_session = sagemaker_session)



### Submit Source Code Change

### Model Evaluation on Dev

### Production Variants Evaluation